In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'prod'
output_dataset = 'de_proto'

In [2]:
import pandas as pd

from IPython.display import display, Markdown

import data_science_dags.configure_warnings

In [3]:
default_query_props = dict(project=project_id, source_dataset=source_dataset)

In [4]:
def printmd(text: str):
    display(Markdown(text))


def read_big_query(query, show_query=True):
    if show_query:
        printmd('> ```sql\n%s\n```' % query.strip())
    return pd.read_gbq(
        query,
        project_id=project_id,
        dialect='standard',
        progress_bar_type='tqdm_notebook'
    )

In [5]:
sample_df = read_big_query(
    '''
    SELECT COUNT(*) AS count
    FROM `{project}.{source_dataset}.mv_Editorial_Person`
    '''.format(
        **default_query_props
    )
)
sample_df.head()

> ```sql
SELECT COUNT(*) AS count
    FROM `elife-data-pipeline.prod.mv_Editorial_Person`
```

,count
0,271391


In [6]:
destination_table = '%s.sample_table' % output_dataset
print('writing to: %s' % destination_table)
sample_df.to_gbq(
    destination_table=destination_table,
    project_id=project_id,
    if_exists='replace',
    progress_bar=False
)

writing to: de_proto.sample_table
